In [14]:
import cv2
import numpy as np
from skimage.feature import hog
from skimage import color, io
from sklearn import svm
from sklearn.model_selection import train_test_split
import os

# Hàm trích xuất đặc trưng HOG từ ảnh
def extract_hog_features(image):
    gray_image = color.rgb2gray(image)  # Chuyển ảnh sang ảnh xám
    fd, hog_image = hog(gray_image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return fd

# Đọc ảnh và trích xuất đặc trưng HOG
data = []
labels = []

# Đường dẫn đến thư mục ảnh
image_folder_board = 'dataSet/board/'
image_folder_non_board = 'dataSet/non_board/'
  # Cập nhật đường dẫn đến thư mục chứa ảnh

# Tải ảnh và gán nhãn (0 cho ảnh không phải bàn cờ, 1 cho ảnh là bàn cờ)
for filename in os.listdir(image_folder_board):
    image = io.imread(os.path.join(image_folder_board, filename))
    features = extract_hog_features(image)
    data.append(features)
    labels.append(1)  # Nhãn 1 cho bàn cờ

for filename in os.listdir(image_folder_non_board):
    image = io.imread(os.path.join(image_folder_non_board, filename))
    features = extract_hog_features(image)
    data.append(features)
    labels.append(0)  # Nhãn 0 cho ảnh không phải bàn cờ


# Chuyển dữ liệu và nhãn thành numpy array
data = np.array(data)
labels = np.array(labels)


In [15]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Khởi tạo mô hình SVM với kernel tuyến tính
model = svm.SVC(kernel='linear')

# Huấn luyện mô hình
model.fit(X_train, y_train)

# Đánh giá mô hình
accuracy = model.score(X_test, y_test)
print(f"Accuracy of the model: {accuracy * 100:.2f}%")

# Lưu mô hình đã huấn luyện
import joblib
joblib.dump(model, 'board_detection_model.pkl')


Accuracy of the model: 100.00%


['board_detection_model.pkl']

In [2]:
import cv2
import numpy as np
from skimage.feature import hog
from skimage import color
import joblib

# Hàm trích xuất đặc trưng HOG từ ảnh
def extract_hog_features(image):
    # Resize ảnh về kích thước cố định (giống tập huấn luyện)
    fixed_size = (300, 300)  # Đảm bảo giống kích thước ảnh huấn luyện
    image = cv2.resize(image, fixed_size)

    # Kiểm tra nếu ảnh có 3 kênh màu (RGB/BGR) thì chuyển sang ảnh xám
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray_image = color.rgb2gray(image)
    else:
        gray_image = image

    # Trích xuất đặc trưng HOG
    fd, hog_image = hog(gray_image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return fd



# Mở camera
cap = cv2.VideoCapture(0)

# Tải mô hình đã huấn luyện
model = joblib.load('board_detection_model.pkl')

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Chuyển đổi ảnh sang xám nếu cần
    if len(frame.shape) == 3 and frame.shape[2] == 3:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        gray = frame  # Nếu đã là ảnh xám, giữ nguyên

    # Trích xuất đặc trưng HOG (đảm bảo resize ảnh trước)
    features = extract_hog_features(gray)

    # Đảm bảo số đặc trưng khớp với mô hình
    if features.shape[0] != model.n_features_in_:
        print(f"⚠️ Cảnh báo: Số đặc trưng ({features.shape[0]}) không khớp với mô hình ({model.n_features_in_})")
        continue  # Bỏ qua frame này để tránh lỗi

    # Dự đoán với mô hình đã huấn luyện
    prediction = model.predict([features])

    # Hiển thị kết quả dự đoán trên màn hình
    label = f"Bàn cờ caro" if prediction == 1 else f"Không phải bàn cờ caro"
    
    # Hiển thị text lên ảnh
    cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Hiển thị video với dự đoán
    cv2.imshow("Board Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Nhấn 'q' để thoát
        break

# Giải phóng tài nguyên
cap.release()
cv2.destroyAllWindows()

